In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever
from langchain_community.retrievers import TFIDFRetriever
from langchain_community.vectorstores import FAISS
from langchain_chroma import Chroma
from langchain_community.embeddings import OllamaEmbeddings

#### Instantiate the embeddings model

In [3]:
embedding = OllamaEmbeddings(model="nomic-embed-text:latest")  # 768 dims

#### Load the previously made vector stores

In [4]:
#TF-IDF
tfidf_retriever = TFIDFRetriever.load_local("tfidf_aoe2.pkl", allow_dangerous_deserialization=True)

#Chroma
chroma_vectorstore = Chroma(embedding_function=embedding, persist_directory="chroma_aoe2")
chroma_retriever = chroma_vectorstore.as_retriever(search_kwargs={"k": 5})

#Faiss
faiss_retriever = FAISS.load_local("faiss_aoe2", embedding, allow_dangerous_deserialization=True).as_retriever()

In [5]:
chroma_retriever.invoke("Villagers")

[Document(metadata={'page': 26, 'source': 'docs/Age_of_Empires_2_Manual.pdf'}, page_content='24 Chapter III  -  Building Your EmpireChapter III\nPutting your villagers to work\nVillagers are invaluable to your civilization. Their primary function\nis to gather wood, food, gold, and stone from the land and deposit it\nin your stockpile. They also construct buildings and repair damaged\nbuildings, boats, and siege weapons. In a pinch, they can evenengage in combat. Fishing Ships also contribute to population count'),
 Document(metadata={'page': 89, 'source': 'docs/Age_of_Empires_2_Manual.pdf'}, page_content='and gold, hunt, forage, fish, herd sheep, and farm. They also construct buildings andrepair damaged buildings, ships, and siege weapons. If necessary, they can also engage in\ncombat. Villager gender is randomly determined when you create a new villager. They\nperform the same tasks regardless of their gender.\nThe great percentage of people in the Middle Ages were\npeasants, serfs, 

In [6]:
chroma_retriever.search_kwargs['k'] = 9

#### Simple querying

In [7]:
def make_llama_3_prompt(user, system="", context=""):
    if system != "":
        system_prompt = (
            f"<|start_header_id|>system<|end_header_id|>\n\n{system}<|eot_id|>"
        )
    return f"<|begin_of_text|>{system_prompt}<|start_header_id|>user<|end_header_id|>\n\n{user}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"

In [8]:
from langchain_core.retrievers import BaseRetriever
from langchain_ollama import ChatOllama
llm = ChatOllama(model="llama3.1:latest")

In [9]:
class BasicAgent:
    def __init__(self, llm:ChatOllama, retriever:BaseRetriever) -> None:
        self.retriever = retriever
        self.llm = llm

    def custom_retriever(self, user_query:str, k):
        try:
            self.retriever.k = k
        except Exception as e:
            self.retriever.search_kwargs['k'] = k
        retrieved_docs = self.retriever.invoke(user_query)
        context = ""
        for doc in retrieved_docs:
            context += f"Extracted from page {doc.metadata['page']} \n{doc.page_content} \n\n"
        return context, retrieved_docs

    def query(self, user_query:str, k=5):
        context, retrieved_docs = self.custom_retriever(user_query, k)
        system_prompt = ("You are helpful assistant, your role is to assist people getting their way around the rules and mechanics of the famous game Age of Empires 2."
                        "You have the task to answer using the following context"
                        f"<CONTEXT>{context}</CONTEXT>"
                        "Keep you answers brief, make reference to the pages used and keep the answer at 50 words at max."
                        "If the answer is not contained in the context, say you don't know")
        prompt = self.make_llama_3_prompt(user_query, system_prompt)
        answer = llm.invoke(prompt)
        return answer.content, context, retrieved_docs
    
    def make_llama_3_prompt(self, user, system="", context=""):
        if system != "":
            system_prompt = (
                f"<|start_header_id|>system<|end_header_id|>\n\n{system}<|eot_id|>"
            )
        return f"<|begin_of_text|>{system_prompt}<|start_header_id|>user<|end_header_id|>\n\n{user}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"

In [10]:
TFIDF_Agent = BasicAgent(llm, tfidf_retriever)
Chroma_Agent = BasicAgent(llm, chroma_retriever)
Faiss_Agent = BasicAgent(llm, faiss_retriever)

#### Manual inspection

In [11]:
answer, context, retrieved_docs = TFIDF_Agent.query("How can I increase the number of villagers?")
print(answer)
retrieved_docs

You can increase the population limit by changing it in the Population box in the pregame settings. (Page 30)


[Document(metadata={'source': 'docs/Age_of_Empires_2_Manual.pdf', 'page': 30}, page_content='The population limit set at the beginning of the game determines how many villagers,\nmilitary units, or ships your civilization can support. The default population limit for mostgames is 75. You can increase or decrease the population limit before you start a game by\nchanging it in the Population  box in the pregame settings. For descriptions of the\nvillagers, military units, and ships you can build, see Chapter VII.'),
 Document(metadata={'source': 'docs/Age_of_Empires_2_Manual.pdf', 'page': 22}, page_content='To change the setting, click the player number. To play a cooperative game, twoor more players must select the same player number and share control of a singlecivilization. Each player can give unrestricted (and even conflicting) orders toall units.\nzTeam  — Shows the players who want to\nstart the game as allies. You can select ateam by clicking the Team  number. A\ndash (-) in the 

In [12]:
answer, context, retrieved_docs = Chroma_Agent.query("How can I increase the number of villagers?")
print(answer)
retrieved_docs

To increase the number of villagers, build a House. (Reference: page 12)


[Document(metadata={'page': 12, 'source': 'docs/Age_of_Empires_2_Manual.pdf'}, page_content='and build new Houses to support them. Put the new villagers to work gatheringmore food and wood. T o start, gather only enough wood for Houses to supportyour villagers and to build the two buildings required to advance to the FeudalAge.\nz Gather food from the forage bushes near your town. Build your Mill near forage\nbushes to gather food faster. Similarly, build Lumber Camps near forests andMining Camps near stone and gold mines.'),
 Document(metadata={'page': 52, 'source': 'docs/Age_of_Empires_2_Manual.pdf'}, page_content='select a villager, and then right-click the expired Farm. Only one\nvillager can work each Farm. Before you can build Farms, you must build a Mill. Farms\ncannot be converted by enemy Monks. You can farm enemy Farms that have been\nabandoned.\nYou can increase the production of your Farms by researching Horse Collar, Heavy Plow,\nand Crop Rotation (at the Mill).'),
 Docume

In [13]:
answer, context, retrieved_docs, Faiss_Agent.query("How can I increase the number of villagers?")
print(answer)
retrieved_docs

To increase the number of villagers, build a House. (Reference: page 12)


[Document(metadata={'page': 12, 'source': 'docs/Age_of_Empires_2_Manual.pdf'}, page_content='and build new Houses to support them. Put the new villagers to work gatheringmore food and wood. T o start, gather only enough wood for Houses to supportyour villagers and to build the two buildings required to advance to the FeudalAge.\nz Gather food from the forage bushes near your town. Build your Mill near forage\nbushes to gather food faster. Similarly, build Lumber Camps near forests andMining Camps near stone and gold mines.'),
 Document(metadata={'page': 52, 'source': 'docs/Age_of_Empires_2_Manual.pdf'}, page_content='select a villager, and then right-click the expired Farm. Only one\nvillager can work each Farm. Before you can build Farms, you must build a Mill. Farms\ncannot be converted by enemy Monks. You can farm enemy Farms that have been\nabandoned.\nYou can increase the production of your Farms by researching Horse Collar, Heavy Plow,\nand Crop Rotation (at the Mill).'),
 Docume

We can see that both FAISS and Chroma retrievers retrieved the same docs while the TF-IDF method didn't

- TF-IDF:
    - 'You can increase the population limit by changing it in the Population box in the pregame settings (page 30).'
- Chroma:
    - 'To increase the number of villagers, build new Houses to support them. (Extracted from page 12)'
- FAISS:
    - 'You can create new villagers by building Houses. The houses will support them. Put the new villagers to work gathering more food and wood. (Page 12)'

It wouldn't be fair to tag the TF-IDF answer as **wrongly answered**. It does answer the questions but the general intention of a player is to actually increase the number of villager in the game, not just by changing a setting.

# Evaluation

We evaluate the quality of the answers given. We won't be evaluating the retriever. Altought in some scenarios it might be a good idea.
Let's go ahead and create an Evaluator, this evaluator will compute the answers for the given questions in the dataset.